In [ ]:
%%sql

-- CONSOLIDATE FABRIC UTILIZATION (BY WORKSPACE) WITH DAILY CAPACITY COST
-- Produces workspace-level allocated cost per capacity per day

-- 1) Base: normalize date and select needed fields from metrics_by_item_and_day
WITH mbid_base AS (
  SELECT
    /* Normalize date: prefer the 'date' column; fall back to 'datetime' if needed */
    COALESCE(
      TO_DATE(metrics_by_item_and_day_date),
      TO_DATE(metrics_by_item_and_day_date, 'yyyy-MM-dd'),
      TO_DATE(metrics_by_item_and_day_date, 'yyyy/MM/dd'),
      TO_DATE(CAST(metrics_by_item_and_day_datetime AS TIMESTAMP))
    ) AS Date,
    metrics_by_item_and_day_capacity_id  AS CapacityId,
    metrics_by_item_and_day_workspace_id AS WorkspaceId,
    CAST(metrics_by_item_and_day_cu__s AS DOUBLE) AS WorkspaceCU
  FROM metrics_by_item_and_day
),

-- 2) Capacities: pick Id/Name; keep distinct rows
caps_norm AS (
  SELECT DISTINCT
    capacities_capacity_id   AS CapacityId,
    capacities_capacity_name AS CapacityName
  FROM capacities
),

-- 3) Workspaces: pick Id/Name; keep distinct rows
ws_norm AS (
  SELECT DISTINCT
    workspaces_workspace_id   AS WorkspaceId,
    workspaces_workspace_name AS WorkspaceName
  FROM workspaces
),

-- 4) Aggregate daily CU per (capacity, workspace)
capacity_ws_day AS (
  SELECT
    CapacityId,
    WorkspaceId,
    Date,
    SUM(WorkspaceCU) AS WorkspaceCU
  FROM mbid_base
  GROUP BY CapacityId, WorkspaceId, Date
),

-- 5) Attach names
capacity_ws_day_named AS (
  SELECT
    d.CapacityId,
    c.CapacityName,
    d.WorkspaceId,
    w.WorkspaceName,
    d.Date,
    d.WorkspaceCU
  FROM capacity_ws_day d
  LEFT JOIN caps_norm c  ON d.CapacityId  = c.CapacityId
  LEFT JOIN ws_norm   w  ON d.WorkspaceId = w.WorkspaceId
),

-- 6) Per-capacity/day totals
cap_day_totals AS (
  SELECT
    CapacityId,
    CapacityName,
    Date,
    SUM(WorkspaceCU) AS CapacityDayTotalCU
  FROM capacity_ws_day_named
  GROUP BY CapacityId, CapacityName, Date
),

-- 7) Final utilization with share
capacity_ws_day_final AS (
  SELECT
    n.CapacityId,
    n.CapacityName,
    n.WorkspaceId,
    COALESCE(n.WorkspaceName,'Unknown') AS WorkspaceName,
    n.Date,
    n.WorkspaceCU,
    t.CapacityDayTotalCU,
    CASE WHEN t.CapacityDayTotalCU > 0 THEN n.WorkspaceCU / t.CapacityDayTotalCU ELSE 0.0 END AS WorkspaceShare
  FROM capacity_ws_day_named n
  LEFT JOIN cap_day_totals t
    ON  n.CapacityId   = t.CapacityId
    AND n.CapacityName = t.CapacityName
    AND n.Date         = t.Date
),

-- 8) Normalize daily capacity COST table
--    - Handles numeric YYYYMMDD or ISO date
--    - Trims/uppercases CapacityName to improve join match
cost_daily_norm AS (
  SELECT
    COALESCE(
      TO_DATE(CAST(date AS STRING), 'yyyyMMdd'),
      TO_DATE(CAST(date AS STRING), 'yyyy-MM-dd'),
      CAST(`date` AS DATE)
    )                                   AS Date,
    UPPER(TRIM(capacityname))           AS CapacityName_norm,
    CAST(cost AS DOUBLE)                AS CapacityCost,
    currency,
    resourceid,
    devicename
  FROM fabric_cost_daily
),

-- 9) Normalize names on the utilization side the same way
util_norm AS (
  SELECT
    Date,
    CapacityId,
    UPPER(TRIM(CapacityName)) AS CapacityName_norm,
    WorkspaceId,
    WorkspaceName,
    WorkspaceCU,
    CapacityDayTotalCU,
    WorkspaceShare
  FROM capacity_ws_day_final
)

-- 10) Join & compute allocated cost per workspace per day
SELECT
  u.Date,
  u.CapacityId,
  -- keep the original friendly CapacityName as well (if you want it):
  -- You can join back to caps_norm if you want the exact casing:
  INITCAP(REPLACE(u.CapacityName_norm, '_', ' ')) AS CapacityName,
  u.WorkspaceId,
  u.WorkspaceName,
  u.WorkspaceCU,
  u.CapacityDayTotalCU,
  u.WorkspaceShare,
  c.CapacityCost,
  ROUND(u.WorkspaceShare * c.CapacityCost, 6) AS AllocatedCost,
  c.currency
FROM util_norm u
LEFT JOIN cost_daily_norm c
  ON  u.Date = c.Date
  AND u.CapacityName_norm = c.CapacityName_norm
WHERE u.WorkspaceCU > 0
ORDER BY u.Date, CapacityName, u.WorkspaceName;
